In [1]:
# Import the custom library
import logic_algorithms as la
import sympy as sp
from sympy.logic.boolalg import Implies, And, Or, Not, Equivalent

# Define the plausability table

In [2]:
# Define the table with the plausibility orders
plausibility_table = {
    'p, q': {'states': ['x'], 'order': 2},
    'p, n_q': {'states': ['y'], 'order': 1},
    'n_p, q': {'states': ['z'], 'order': 2},
    'n_p, n_q': {'states': ['w'], 'order': 3}
}

# Print the table to verify
for key, value in plausibility_table.items():
    print(f"{key}: {value}")


p, q: {'states': ['x'], 'order': 2}
p, n_q: {'states': ['y'], 'order': 1}
n_p, q: {'states': ['z'], 'order': 2}
n_p, n_q: {'states': ['w'], 'order': 3}


# Contraction

In [3]:
# Results of the contraction
print("Contraction with p: " + str(la.contraction(plausibility_table, 'p')))
print("Contraction with q: " + str(la.contraction(plausibility_table, 'q')))
print("Contraction with n_p: " + str(la.contraction(plausibility_table, 'n_p')))
print("Contraction with n_q: " + str(la.contraction(plausibility_table, 'n_q')))

TypeError: unsupported operand type(s) for &: 'bool' and 'str'

# Revision

In [ ]:
# Results of the revision
print("Revision with p: " + str(la.revision(plausibility_table, 'p')))
print("Revision with q: " + str(la.revision(plausibility_table, 'q')))
print("Revision with n_p: " + str(la.revision(plausibility_table, 'n_p')))
print("Revision with n_q: " + str(la.revision(plausibility_table, 'n_q')))

Revision with p: ['y']
Revision with q: ['x']
Revision with n_p: ['z']
Revision with n_q: ['y']


# Check if a formula is on the belief set after revision with a proposition

In [ ]:
# Define the symbols
p, q = sp.symbols('p q')

# Define the formulas
formulas = {
    'p -> q': sp.Implies(p, q),
    'q -> p': sp.Implies(q, p),
    'p ∧ q': sp.And(p, q),
    'p ∨ q': sp.Or(p, q),
    'p ↔ q': sp.Equivalent(p, q)
}

# Given belief set revision with ¬p
belief_revision = sp.Not(p)

# Check each formula against the revised belief set
results = {formula: sp.simplify(sp.Implies(belief_revision, expr)) for formula, expr in formulas.items()}

# Print the results
print("Results of the belief revision with ¬p (simplification):")
for formula, result in results.items():
    print(f"{formula}: {result}")

Results of the belief revision with ¬p (simplification):
p -> q: True
q -> p: p | ~q
p ∧ q: p
p ∨ q: p | q
p ↔ q: p | ~q


In [ ]:
p, q = sp.symbols('p q')
belief_base = {p, q, Implies(p, q), Implies(Not(p), q)}
proposition = q

result = la.bb_contraction(belief_base, proposition)
for i, subset in enumerate(result):
    print(f"Maximal subset {i+1}: {set(subset)}")

Maximal subset 1: {Implies(p, q)}
Maximal subset 2: {p, Implies(~p, q)}
